In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
cd /home/juliakhalina/notebooks

/home/juliakhalina/notebooks


In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 19:59:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/03 19:59:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [6]:
import pandas as pd

In [7]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-06.csv')

In [8]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [9]:
!head -n 10 fhvhv_tripdata_2021-06.csv

dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,
B02510,2021-06-01 00:18:15,2021-06-01 00:25:47,49,17,N,
B02510,2021-06-01 00:33:06,2021-06-01 00:42:46,49,225,N,
B02510,2021-06-01 00:46:27,2021-06-01 00:56:50,225,177,N,
B02764,2021-06-01 00:48:06,2021-06-01 01:04:10,209,45,N,B02764


In [10]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [11]:
df = df.repartition(12)

In [12]:
df.write.parquet('fhvhv/2021/06/')

AnalysisException: path file:/home/juliakhalina/notebooks/fhvhv/2021/06 already exists.

In [13]:
df = spark.read.parquet('fhvhv/2021/06/')

In [14]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: string, DOLocationID: string, SR_Flag: string, Affiliated_base_number: string]

In [15]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [17]:
df \
    .select('pickup_datetime') \
    .filter(df.pickup_datetime.like('2021-06-15%')) \
    .count()

452470

In [21]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [22]:
from pyspark.sql import types

In [24]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.IntegerType(), True)
    ])

In [25]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

In [26]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                  null|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                  null|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                  null|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                  null|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [30]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', IntegerType(), True)])

In [41]:
df_res = \
    (df.dropoff_datetime - df.pickup_datetime)
df.filter(df_res).show()

AnalysisException: filter expression '(dropoff_datetime - pickup_datetime)' of type interval day to second is not a boolean.;
Filter (dropoff_datetime#106 - pickup_datetime#105)
+- Relation [dispatching_base_num#104,pickup_datetime#105,dropoff_datetime#106,PULocationID#107,DOLocationID#108,SR_Flag#109,Affiliated_base_number#110] csv
